In [83]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, split

sc = pyspark.SparkContext('local[*]')

sqlContext = SQLContext(sc)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-b1129a2099c1>:5 

In [123]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('Arizona_10users.csv')
df.columns

['user_id', 'business_reviewed_count', 'average_stars']

In [124]:

# one_user_df = df.first()
# one_user_df

In [125]:
# userId = one_user_df["user_id"]
# type(one_user_df)

In [126]:
first_five = df.limit(10)
first_five

DataFrame[user_id: string, business_reviewed_count: int, average_stars: double]

In [127]:
# first_five = first_five.rdd.filter(lambda x: x != one_user_df)

In [128]:
new = first_five.rdd.map(lambda x: (x[0],x[1],x[2]))

In [129]:
# new.collect()

In [130]:
newDf = new.toDF()
newDf['_2','_3'].describe().show()

+-------+-----------------+-----------------+
|summary|               _2|               _3|
+-------+-----------------+-----------------+
|  count|               10|               10|
|   mean|             14.3|            3.412|
| stddev|9.006787563955431|0.977716159674621|
|    min|                6|             2.29|
|    max|               31|              4.9|
+-------+-----------------+-----------------+



In [133]:
normalized_rdd = newDf.rdd.map(lambda x: (x[0],((x[1] - 14.3)/9.01),((x[2] - 3.41)/0.98)))

In [134]:
normalized_rdd.collect()

[(u'-55DgUo52I3zW9RxkZ-EAQ', 1.2985571587125415, 1.1734693877551015),
 (u'-inp099-1gsJF3KPaS-mbg', 0.41065482796892333, 0.2653061224489794),
 (u'0JaDPuKP812uXA6VIDMPZA', -0.921198668146504, -1.142857142857143),
 (u'1M5EuCw_7tJjJVmmnLmR9g', -0.921198668146504, -0.5306122448979592),
 (u'1Q-tjPYDBaFZJouj8Ynl2g', 0.632630410654828, 1.5204081632653064),
 (u'1jtr65gmv4A3uqCDaaXcxQ', -0.921198668146504, -0.41836734693877564),
 (u'2OtKt_7AZkfPXioU317R7g', 1.8534961154273029, 1.2959183673469383),
 (u'2zqSXXGVGmTWZ86mAq-69w', -0.6992230854605994, -1.1020408163265307),
 (u'3UA89mI2pN2KphXN3QXPIQ', -0.3662597114317426, -0.5408163265306125),
 (u'3g-2-nE22LxldS3Rx5IzNg', -0.3662597114317426, -0.5000000000000002)]

In [135]:
first_user = normalized_rdd.first()
filtered_norm_users = normalized_rdd.filter(lambda x: x != first_user)

In [139]:
first_user

(u'-55DgUo52I3zW9RxkZ-EAQ', 1.2985571587125415, 1.1734693877551015)

In [144]:
import math

def cosine_sim(user):
    user = list(user)
    num = 0
    den1 = 0
    den2 = 0
    for i in range(1,len(user)):
        num += user[i] * first_user[i]
        den1 += user[i] * user[i]
        den2 += first_user[i] * first_user[i]
    den1 = math.sqrt(den1)
    den2 = math.sqrt(den2)
    cos_sim = num / float(den1 * den2)
    return cos_sim

corr = filtered_norm_users.map(lambda x : (x[0], x[1], x[2], cosine_sim(x)))

In [145]:
corr.collect()

[(u'-inp099-1gsJF3KPaS-mbg',
  0.41065482796892333,
  0.2653061224489794,
  0.9870286235232484),
 (u'0JaDPuKP812uXA6VIDMPZA',
  -0.921198668146504,
  -1.142857142857143,
  -0.9876165388731567),
 (u'1M5EuCw_7tJjJVmmnLmR9g',
  -0.921198668146504,
  -0.5306122448979592,
  -0.9775587683080643),
 (u'1Q-tjPYDBaFZJouj8Ynl2g',
  0.632630410654828,
  1.5204081632653064,
  0.904045678062686),
 (u'1jtr65gmv4A3uqCDaaXcxQ',
  -0.921198668146504,
  -0.41836734693877564,
  -0.9527792155191327),
 (u'2OtKt_7AZkfPXioU317R7g',
  1.8534961154273029,
  1.2959183673469383,
  0.9922390243234007),
 (u'2zqSXXGVGmTWZ86mAq-69w',
  -0.6992230854605994,
  -1.1020408163265307,
  -0.9636190150972948),
 (u'3UA89mI2pN2KphXN3QXPIQ',
  -0.3662597114317426,
  -0.5408163265306125,
  -0.9711778070265059),
 (u'3g-2-nE22LxldS3Rx5IzNg',
  -0.3662597114317426,
  -0.5000000000000002,
  -0.9793173099554092)]